In [ ]:
# Predicting Employee Retention Using Logistic Regression

# 📘 1. Load Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

# 📥 2. Load Dataset
df = pd.read_csv('employee_data.csv')  # Replace with your actual dataset name
print("Initial Data Preview:")
df.head()

# 🧹 3. Data Cleaning
df.drop(['EmployeeNumber', 'Over18', 'StandardHours'], axis=1, inplace=True)  # Drop constant/irrelevant columns
print("\nMissing values:")
print(df.isnull().sum())

# 🔍 4. Exploratory Data Analysis (EDA)
sns.countplot(x='Attrition', data=df)
plt.title('Attrition Distribution')
plt.show()

numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols].hist(bins=15, figsize=(15, 10))
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,8))
sns.heatmap(df[numeric_cols].corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# 🛠 5. Feature Engineering
df = pd.get_dummies(df, drop_first=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop('Attrition_Yes', axis=1))
X = pd.DataFrame(X_scaled, columns=df.drop('Attrition_Yes', axis=1).columns)
y = df['Attrition_Yes']

# 🔀 6. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 🔁 7. Feature Selection with RFE
model = LogisticRegression(max_iter=1000)
rfe = RFE(model, n_features_to_select=15)
rfe.fit(X_train, y_train)
selected_features = X_train.columns[rfe.support_]
print("Selected Features:", selected_features)

# ⚙️ 8. Logistic Regression Model
final_model = LogisticRegression()
final_model.fit(X_train[selected_features], y_train)

# 📈 9. Evaluation
y_pred = final_model.predict(X_test[selected_features])
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, final_model.predict_proba(X_test[selected_features])[:,1])
plt.plot(fpr, tpr, label='ROC Curve')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

# ✅ 10. Summary Output Placeholder
# Replace the XX% placeholders in your report manually after running this notebook and reading the output metrics
